# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Jun 13 2022 10:24AM  239440        10  0085755314   
1  Jun 13 2022 10:24AM  239440        10  0085755464   
2  Jun 13 2022 10:24AM  239440        10  0085755803   
3  Jun 13 2022 10:24AM  239440        10  0085755839   
4  Jun 13 2022 10:24AM  239440        10  0085759497   
5  Jun 13 2022 10:24AM  239440        10  0085759494   
6  Jun 13 2022 10:24AM  239440        10  0085759498   
7  Jun 13 2022 10:24AM  239440        10  0085759496   
8  Jun 13 2022 10:24AM  239443        19     8139058   
9  Jun 13 2022 10:24AM  239441        15     8138381   

                     Customer ShipmentStatus  
0           ISDIN Corporation       Released  
1           ISDIN Corporation       Released  
2           ISDIN Corporation       Released  
3           ISDIN Corporation       Released  
4           ISDIN Corporation       Released  
5           ISDIN Corporation       Released  
6           ISDIN Corporation       Released  
7           ISDIN Corporation       Released  
8              Graystone, LLC       Released  
9  Laser Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
33  239438       Released         47
34  239439       Released          1
35  239440       Released          8
36  239441       Released          7
37  239443       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
239438                NaN      47.0
239439                NaN       1.0
239440                NaN       8.0
239441                NaN       7.0
239443                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
239387               20.0       1.0
239388                0.0       1.0
239389                0.0       1.0
239391               14.0       1.0
239392                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
239387                 20         1
239388                  0         1
239389                  0         1
239391                 14         1
239392                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               239387         20         1
1               239388          0         1
2               239389          0         1
3               239391         14         1
4               239392          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               239387        20        1
1               239388                  1
2               239389                  1
3               239391        14        1
4               239392                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0    Jun 13 2022 10:24AM  239440        10                ISDIN Corporation
8    Jun 13 2022 10:24AM  239443        19                   Graystone, LLC
9    Jun 13 2022 10:24AM  239441        15       Laser Pharmaceuticals, LLC
16   Jun 13 2022 10:20AM  239439        19                 SD Head USA, LLC
17   Jun 13 2022 10:18AM  239438        10                ISDIN Corporation
64   Jun 13 2022 10:13AM  239433        15  Brookfield Pharmaceuticals, LLC
127  Jun 13 2022 10:12AM  239428        10                ISDIN Corporation
167  Jun 13 2022 10:11AM  239436        10        Beach Patient Assistance 
169  Jun 13 2022 10:09AM  239434        19            ACG North America LLC
170  Jun 13 2022 10:06AM  239432        15            Alliance Pharma, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Jun 13 2022 10:24AM  239440        10                ISDIN Corporation   
1  Jun 13 2022 10:24AM  239443        19                   Graystone, LLC   
2  Jun 13 2022 10:24AM  239441        15       Laser Pharmaceuticals, LLC   
3  Jun 13 2022 10:20AM  239439        19                 SD Head USA, LLC   
4  Jun 13 2022 10:18AM  239438        10                ISDIN Corporation   
5  Jun 13 2022 10:13AM  239433        15  Brookfield Pharmaceuticals, LLC   
6  Jun 13 2022 10:12AM  239428        10                ISDIN Corporation   
7  Jun 13 2022 10:11AM  239436        10        Beach Patient Assistance    
8  Jun 13 2022 10:09AM  239434        19            ACG North America LLC   
9  Jun 13 2022 10:06AM  239432        15            Alliance Pharma, Inc.   

  Executing Released  
0                  8  
1                  1  
2                  7  
3                  1  
4                 47  
5                 63  
6                 40  
7                  2  
8                  1  
9                 23

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Jun 13 2022 10:24AM  239440        10                ISDIN Corporation   
1  Jun 13 2022 10:24AM  239443        19                   Graystone, LLC   
2  Jun 13 2022 10:24AM  239441        15       Laser Pharmaceuticals, LLC   
3  Jun 13 2022 10:20AM  239439        19                 SD Head USA, LLC   
4  Jun 13 2022 10:18AM  239438        10                ISDIN Corporation   
5  Jun 13 2022 10:13AM  239433        15  Brookfield Pharmaceuticals, LLC   
6  Jun 13 2022 10:12AM  239428        10                ISDIN Corporation   
7  Jun 13 2022 10:11AM  239436        10        Beach Patient Assistance    
8  Jun 13 2022 10:09AM  239434        19            ACG North America LLC   
9  Jun 13 2022 10:06AM  239432        15            Alliance Pharma, Inc.   

  Released Executing  
0        8            
1        1            
2        7            
3        1            
4       47            
5       63            
6       40            
7        2            
8        1            
9       23

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                    Customer Released  \
0  Jun 13 2022 10:24AM  239440        10           ISDIN Corporation        8   
1  Jun 13 2022 10:24AM  239443        19              Graystone, LLC        1   
2  Jun 13 2022 10:24AM  239441        15  Laser Pharmaceuticals, LLC        7   
3  Jun 13 2022 10:20AM  239439        19            SD Head USA, LLC        1   
4  Jun 13 2022 10:18AM  239438        10           ISDIN Corporation       47   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                    Customer  \
0  Jun 13 2022 10:24AM  239440        10           ISDIN Corporation   
1  Jun 13 2022 10:24AM  239443        19              Graystone, LLC   
2  Jun 13 2022 10:24AM  239441        15  Laser Pharmaceuticals, LLC   
3  Jun 13 2022 10:20AM  239439        19            SD Head USA, LLC   
4  Jun 13 2022 10:18AM  239438        10           ISDIN Corporation   

   Released  Executing  
0       8.0        NaN  
1       1.0        NaN  
2       7.0        NaN  
3       1.0        NaN  
4      47.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                    Customer  \
0  Jun 13 2022 10:24AM  239440        10           ISDIN Corporation   
1  Jun 13 2022 10:24AM  239443        19              Graystone, LLC   
2  Jun 13 2022 10:24AM  239441        15  Laser Pharmaceuticals, LLC   
3  Jun 13 2022 10:20AM  239439        19            SD Head USA, LLC   
4  Jun 13 2022 10:18AM  239438        10           ISDIN Corporation   

   Released  Executing  
0       8.0        0.0  
1       1.0        0.0  
2       7.0        0.0  
3       1.0        0.0  
4      47.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3351892     396.0       17.0
12          239425      32.0        2.0
15          718306      93.0        0.0
16          239407       0.0        1.0
18         1436432       6.0        0.0
19         1197118       5.0       20.0
20          478808      15.0        0.0
22          478777       2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3351892     396.0       17.0
1        12   239425      32.0        2.0
2        15   718306      93.0        0.0
3        16   239407       0.0        1.0
4        18  1436432       6.0        0.0
5        19  1197118       5.0       20.0
6        20   478808      15.0        0.0
7        22   478777       2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     396.0       17.0
1        12      32.0        2.0
2        15      93.0        0.0
3        16       0.0        1.0
4        18       6.0        0.0
5        19       5.0       20.0
6        20      15.0        0.0
7        22       2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  396.0
1        12  Released   32.0
2        15  Released   93.0
3        16  Released    0.0
4        18  Released    6.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   16   18    19    20   22
Status                                                 
Executing   17.0   2.0   0.0  1.0  0.0  20.0   0.0  0.0
Released   396.0  32.0  93.0  0.0  6.0   5.0  15.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   16   18    19    20   22
0          Executing   17.0   2.0   0.0  1.0  0.0  20.0   0.0  0.0
1           Released  396.0  32.0  93.0  0.0  6.0   5.0  15.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   16   18    19    20   22
0  Executing   17.0   2.0   0.0  1.0  0.0  20.0   0.0  0.0
1   Released  396.0  32.0  93.0  0.0  6.0   5.0  15.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()